### TODO This notebook is under construction, not ready yet

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
N = 30
x_data = (np.linspace(0,10,N)).astype('float32')
y_data = (2.42 * x_data + 0.42 + np.random.normal(0,1,N)).astype('float32')

## Writing the definition file

When using checkpointing, one is able to write out a the weights of the graph and **not** the definition of the graph. 

In [2]:
a = tf.Variable(1.0, name = 'a') #Note that 1.0 is needed
b = tf.Variable(0.01, name = 'b')
x = tf.placeholder('float32', [N], name='x_data')
y = tf.placeholder('float32', [N], name='y_data')


resi = a*x + b - y
loss = tf.reduce_sum(tf.square(resi), name='loss') 
init_op = tf.initialize_all_variables() #Initialization op 
train_op = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

In [ ]:
epochs = 1000
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init_op)
    for e in range(5): 
        sess.run(train_op, feed_dict={x:x_data, y:y_data})
    res = sess.run([loss, a, b], feed_dict={x:x_data, y:y_data})
    print(res)
    save_path = saver.save(sess, "checkpoints/model.ckpt")
    print("Model saved in file: %s" % save_path)
!ls -l checkpoints

In [ ]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "checkpoints/model.ckpt")
    res = sess.run([loss, a, b], feed_dict={x:x_data, y:y_data})
    print(res)

## Saving and loading the graph (a bit more advanced and still not working)
We now want to save the graph (i.e. the structure and not the weights). The graphs are store into so-called protobuffers, which can be written as text.


The current graph we are using is the default graph. Let's get the definition:

In [ ]:
definition = tf.get_default_graph().as_graph_def()
type(definition)

In [ ]:
tf.train.write_graph(definition, "graphdef/", 'test.pb', as_text=True)
%ls -rtl graphdef
definition.ByteSize()

In [ ]:
tf.reset_default_graph() # Clearing the default graph
#saver = tf.train.Saver() # Not working since the graph is empty

In [ ]:
from google.protobuf import text_format

# Loading the graph definition
graph_def = tf.GraphDef() 
with open('graphdef/test.pb', "rb") as f:
    text_format.Merge(f.read(), graph_def)
    # Setting the graph definition 
print(graph_def.ByteSize())
with tf.Session() as sess:
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
    sess.run(init_op)
    print(tf.all_variables())
    print(sess.graph.get_all_collection_keys())
    saver = tf.train.Saver()
    saver.restore(sess, "checkpoints/model.ckpt")
    res = sess.run([loss, a, b], feed_dict={x:x_data, y:y_data})
    print(res)

### Freezing the graph

Here we want to write out the graph and the weights, with is quite convenient if you e.g. want to deploy for model e.g. on a Android Device. We have to **transfer the variables to constants with the same value** in that case, since for variables it is stated that " they're held in separate checkpoint files, and there are Variable ops in the graph that load the latest values when they're initialized". See [https://www.tensorflow.org/versions/r0.7/how_tos/tool_developers/index.html#freezing](freezing)


For freezing a graph, we have to have an intialized graph


In [ ]:
from tensorflow.python.client import graph_util
with tf.Session() as sess:
    
    output_graph_def = graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), ['loss', 'y'])
    print('Finished all')

In [ ]:
type(['Hallo'])